In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
from functools import partial
import os

todo_items = []

# 레이아웃
layout_half = widgets.Layout(width='45%')
layout_full = widgets.Layout(width='100%')
layout_button = widgets.Layout(width='120px', height='32px')
layout_wide_button = widgets.Layout(width='240px', height='32px')
layout_label = widgets.Layout(width='70px')

# 전체 진행률
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, style={'bar_color': 'black'}, layout=widgets.Layout(width='50%', height='20px'))
overall_progress_label = widgets.Label(value='(0%)', layout=widgets.Layout(margin='0 10px'))

todo_list_box = widgets.HBox([], layout=layout_full)

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list():
    rows = []
    for item in todo_items:
        title = widgets.Label(value=f"할 일: {item['title']}", layout=layout_full)
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_full)
        percent = widgets.Label(value=f"{item['progress']}%", layout=layout_label)
        def on_progress_change(change, i=item, label=percent):
            i['progress'] = change['new']
            label.value = f"{change['new']}%"
            update_overall_progress()

        progress_slider.observe(on_progress_change, names='value')
        progress_row = widgets.HBox([widgets.Label('진행률', layout=layout_label), progress_slider, percent])

        due_label = widgets.Label(f"마감일: {item['due'].strftime('%Y-%m-%d') if item['due'] else '-'}", layout=layout_full)
        priority_label = widgets.Label(f"중요도: {item['priority']}", layout=layout_full)

        complete_btn = widgets.Button(description='완료✅', layout=layout_button, style={'button_color': '#d3d3d3'})
        delete_btn = widgets.Button(description='삭제❌', layout=layout_button, style={'button_color': '#d3d3d3'})
        def on_complete(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()
        def on_delete(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()
        complete_btn.on_click(partial(on_complete, i=item))
        delete_btn.on_click(partial(on_delete, i=item))
        button_row = widgets.HBox([complete_btn, delete_btn], layout=layout_full)

        one_card = widgets.VBox([title, progress_row, due_label, priority_label, button_row], layout=layout_half)
        rows.append(one_card)
    todo_list_box.children = rows

# 입력
title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_half)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_half)
due_input = widgets.DatePicker(layout=layout_half)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_half)
add_button = widgets.Button(description='추가', button_style='success', layout=layout_wide_button)

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([{**i, 'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None} for i in todo_items], f)

def load_data(b):
    try:
        if os.path.exists('todo_data.json'):
            with open('todo_data.json', 'r') as f:
                raw = json.load(f)
                todo_items.clear()
                for i in raw:
                    todo_items.append({
                        'title': i['title'],
                        'progress': i['progress'],
                        'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                        'priority': i.get('priority', '보통')
                    })
                update_overall_progress()
                refresh_list()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_button)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_button)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# 출력
display(widgets.VBox([
    widgets.Label(value='DAVID mk16 - UI 개선 통합버전'),
    widgets.HBox([widgets.Label('할 일'), title_input]),
    widgets.HBox([widgets.Label('진행률'), progress_input]),
    widgets.HBox([widgets.Label('마감일'), due_input]),
    widgets.HBox([widgets.Label('중요도'), priority_input]),
    widgets.HBox([add_button], layout=widgets.Layout(justify_content='center', padding='10px')),
    widgets.HBox([save_btn, load_btn], layout=widgets.Layout(justify_content='center', padding='10px')),
    widgets.HBox([overall_progress_bar, overall_progress_label], layout=widgets.Layout(justify_content='center')),
    todo_list_box
]))

update_overall_progress()
refresh_list()
